<a href="https://colab.research.google.com/github/AliAbdallah21/Advanced-Retrievers-in-LlamaIndex/blob/main/Explore_Advanced_Retrievers_in_LlamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install llama-index-core \
    llama-index-llms-openai \
    llama-index-embeddings-openai \
    llama-index-retrievers-bm25 \
    sentence-transformers \
    rank-bm25 \
    pystemmer \
    llama-index-embeddings-huggingface

In [36]:
# @title imports
import os
import json
import openai
from typing import List, Dict
import asyncio
import warnings
warnings.filterwarnings("ignore")
from google.colab import userdata


#LLamaIndex imports

from llama_index.core import(
    VectorStoreIndex,
    SimpleDirectoryReader,
    Document,
    Settings,
    DocumentSummaryIndex,
    SummaryIndex,
    KeywordTableIndex
)


from llama_index.core.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
    AutoMergingRetriever,
    RecursiveRetriever,
    QueryFusionRetriever
)

from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexLLMRetriever,
    DocumentSummaryIndexEmbeddingRetriever,
)

from llama_index.core.node_parser import SentenceSplitter, HierarchicalNodeParser
from llama_index.core.schema import NodeWithScore, QueryBundle
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core.embeddings import BaseEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI # Import OpenAI class
from llama_index.core import StorageContext
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.vector_stores import SimpleVectorStore



# Advanced retriever imports
from llama_index.retrievers.bm25 import BM25Retriever
from sentence_transformers import SentenceTransformer

# Statistical libraries for fusion techniques
try:
    from scipy import stats
    SCIPY_AVAILABLE = True
except ImportError:
    SCIPY_AVAILABLE = False
    print("⚠️ scipy not available - some advanced fusion features will be limited")

print("✅ All imports successful!")



✅ All imports successful!


In [ ]:
# @title OpenAi() key setup
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
# @title Initialize Embedding Model
print("🔧 Initializing HuggingFace embeddings...")
embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)
print("✅ HuggingFace embeddings initialized!")

# Setup with watsonx.ai
print("🔧 Initializing openai LLM...")

llm = OpenAI()  # Instantiate the OpenAI class

# Configure global settings
Settings.llm = llm
Settings.embed_model = embed_model
print("✅ OpenAI LLM and embeddings configured!")

🔧 Initializing HuggingFace embeddings...
✅ HuggingFace embeddings initialized!
🔧 Initializing openai LLM...
✅ OpenAI LLM and embeddings configured!


In [ ]:
# @title Sample data
# Sample data for the lab - AI/ML focused documents
SAMPLE_DOCUMENTS = [
    "Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn from data.",
    "Deep learning uses neural networks with multiple layers to model and understand complex patterns in data.",
    "Natural language processing enables computers to understand, interpret, and generate human language.",
    "Computer vision allows machines to interpret and understand visual information from the world.",
    "Reinforcement learning is a type of machine learning where agents learn to make decisions through rewards and penalties.",
    "Supervised learning uses labeled training data to learn a mapping from inputs to outputs.",
    "Unsupervised learning finds hidden patterns in data without labeled examples.",
    "Transfer learning leverages knowledge from pre-trained models to improve performance on new tasks.",
    "Generative AI can create new content including text, images, code, and more.",
    "Large language models are trained on vast amounts of text data to understand and generate human-like text."
]

# Consistent query examples used throughout the lab
DEMO_QUERIES = {
    "basic": "What is machine learning?",
    "technical": "neural networks deep learning",
    "learning_types": "different types of learning",
    "advanced": "How do neural networks work in deep learning?",
    "applications": "What are the applications of AI?",
    "comprehensive": "What are the main approaches to machine learning?",
    "specific": "supervised learning techniques"
}

print(f"📄 Loaded {len(SAMPLE_DOCUMENTS)} sample documents")
print(f"🔍 Prepared {len(DEMO_QUERIES)} consistent demo queries")
for i, doc in enumerate(SAMPLE_DOCUMENTS[:3], 1):
    print(f"{i}. {doc}")
print("...")

📄 Loaded 10 sample documents
🔍 Prepared 7 consistent demo queries
1. Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn from data.
2. Deep learning uses neural networks with multiple layers to model and understand complex patterns in data.
3. Natural language processing enables computers to understand, interpret, and generate human language.
...


In [ ]:
# @title Initiallizing Lab Class
class AdvancedRetrievalsLab:
    def __init__(self):
      print("🔧 Initializing Advanced Retrievals Lab...")
      self.documents = [Document(text=text) for text in SAMPLE_DOCUMENTS]
      self.nodes = SentenceSplitter().get_nodes_from_documents(self.documents)


      print("📊 Creating indexes...")
      self.vector_index = VectorStoreIndex(self.nodes)
      self.document_summary_index = DocumentSummaryIndex.from_documents(self.documents)
      self.keyword_table_index = KeywordTableIndex.from_documents(self.documents)
      print("✅ Advanced Retrievers Lab Initialized!")
      print(f"📄 Loaded {len(self.documents)} documents")
      print(f"🔢 Created {len(self.nodes)} nodes")



lab = AdvancedRetrievalsLab()

🔧 Initializing Advanced Retrievals Lab...
📊 Creating indexes...
current doc id: 2464019a-ee5c-4609-b2d3-303d0347c461
current doc id: 6b46271b-8ce9-4484-af3f-4cb52d6343e7
current doc id: 866aa372-7918-43e2-bd14-2100b2182900
current doc id: a5787c28-8f50-4e6a-92b2-4bf574253c41
current doc id: 88646138-4dbc-4377-adb9-dd7bb2766f88
current doc id: 093d3dfc-2925-4fb3-8c86-687bd5c7d700
current doc id: ca367a00-f82e-4a24-bf35-d73325b51daa
current doc id: ed72e3ff-3527-4c16-89e4-daf6bea9f893
current doc id: 57824d2f-b511-46b1-9667-a3df6d9a4489
current doc id: d0bf3f6a-472a-4e14-a3d8-8a5880c39140
✅ Advanced Retrievers Lab Initialized!
📄 Loaded 10 documents
🔢 Created 10 nodes


In [ ]:
# @title Vector Index Retriever
print ("=" * 60)
print ("🔧 Vector Index Retriever")
print ("=" * 60)

vector_retriever = VectorIndexRetriever(
    index=lab.vector_index,
    similarity_top_k=3
)

# Another way of retrieval
# alt_retriever = lab.vector_index.as_retriever(similarity_top_k = 3)

query = DEMO_QUERIES["basic"]
nodes = vector_retriever.retrieve(query)

print(f"Query: {query}")
print(f"Retrieved {len(nodes)} nodes:")
for i, node in enumerate(nodes,1):
  print(f"{i}. Score: {node.score:.4f}")
  print(f"    Text: {node.text[:100]}...")
  print()

🔧 Vector Index Retriever
Query: What is machine learning?
Retrieved 3 nodes:
1. Score: 0.8700
    Text: Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn fr...

2. Score: 0.7644
    Text: Reinforcement learning is a type of machine learning where agents learn to make decisions through re...

3. Score: 0.6979
    Text: Supervised learning uses labeled training data to learn a mapping from inputs to outputs....



In [ ]:
# @title BM25 Retriever - Advanced Keyword-Based Search
print ("=" * 60)
print ("🔧 BM25 Retriever")
print ("=" * 60)

try:
  import Stemmer

  bm_25_retriever = BM25Retriever.from_defaults(
      nodes = lab.nodes,
      similarity_top_k = 3,
      stemmer = Stemmer.Stemmer('english'),
      language = "english"
  )

  query = DEMO_QUERIES["technical"]
  nodes = bm_25_retriever.retrieve(query)

  print (f"Query: {query}")
  print("BM25 analyzes exact keyword matches with sophisticated scoring")
  print (f"Retrieved {len(nodes)} nodes:")

  for i, node in enumerate(nodes , 1):
    score = node.score if hasattr(node, 'score') and node.score else 0
    print(f"{i}. BM25 Score: {score:.4f}")
    print(f"   Text: {node.text[:100]}...")


    text_lower = node.text.lower()
    query_terms = query.lower().split()
    found_terms = [term for term in query_terms if term in text_lower]
    if found_terms:
      print(f"   Matched terms: {found_terms}")
    print()


  print("BM25 vs TF-IDF Comparison:")
  print("TF-IDF Problem: Linear term frequency scaling")
  print("  Example: 10 occurrences → score of 10, 100 occurrences → score of 100")
  print("BM25 Solution: Saturation function")
  print("  Example: 10 occurrences → high score, 100 occurrences → slightly higher score")
  print()
  print("TF-IDF Problem: No document length consideration")
  print("  Example: Long documents dominate results")
  print("BM25 Solution: Length normalization (b parameter)")
  print("  Example: Scores adjusted based on document length vs. average")
  print()
  print("Key BM25 Parameters:")
  print("- k1 ≈ 1.2: Term frequency saturation (how quickly scores plateau)")
  print("- b ≈ 0.75: Document length normalization (0=none, 1=full)")
  print("- IDF weighting: Rare terms get higher scores")


except ImportError:

  print("⚠️ BM25Retriever requires 'pip install PyStemmer'")
  print("Demonstrating BM25 concepts with fallback vector search...")
  fallback_retriever = lab.vector_index.as_retriever(similarity_top_k=3)
  query = DEMO_QUERIES["technical"]
  nodes = fallback_retriever.retrieve(query)

  print(f"Query: {query}")
  print("(Using vector fallback to demonstrate BM25 concepts)")

  for i, node in enumerate(nodes, 1):
      print(f"{i}. Vector Score: {node.score:.4f}")
      print(f"   Text: {node.text[:100]}...")

      # Demonstrate TF-IDF concept manually
      text_lower = node.text.lower()
      query_terms = query.lower().split()
      found_terms = [term for term in query_terms if term in text_lower]

      if found_terms:
          print(f"   → BM25 would boost this result for terms: {found_terms}")
      print()

  print("BM25 Concept Demonstration:")
  print("1. TF-IDF Foundation:")
  print("   - Term Frequency: How often words appear in document")
  print("   - Inverse Document Frequency: How rare words are across collection")
  print("   - TF-IDF = TF × IDF (balances frequency vs rarity)")
  print()
  print("2. BM25 Improvements:")
  print("   - Saturation: Prevents over-scoring repeated terms")
  print("   - Length normalization: Prevents long document bias")
  print("   - Tunable parameters: k1 (saturation) and b (length adjustment)")
  print()
  print("3. Real-world Usage:")
  print("   - Elasticsearch default scoring function")
  print("   - Apache Lucene/Solr standard")
  print("   - Used in 83% of text-based recommender systems")
  print("   - Developed by Robertson & Spärck Jones at City University London")





DEBUG:bm25s:Building index from IDs objects


🔧 BM25 Retriever
Query: neural networks deep learning
BM25 analyzes exact keyword matches with sophisticated scoring
Retrieved 3 nodes:
1. BM25 Score: 2.5203
   Text: Deep learning uses neural networks with multiple layers to model and understand complex patterns in ...
   Matched terms: ['neural', 'networks', 'deep', 'learning']

2. BM25 Score: 0.3372
   Text: Reinforcement learning is a type of machine learning where agents learn to make decisions through re...
   Matched terms: ['learning']

3. BM25 Score: 0.3024
   Text: Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn fr...
   Matched terms: ['learning']

BM25 vs TF-IDF Comparison:
TF-IDF Problem: Linear term frequency scaling
  Example: 10 occurrences → score of 10, 100 occurrences → score of 100
BM25 Solution: Saturation function
  Example: 10 occurrences → high score, 100 occurrences → slightly higher score

TF-IDF Problem: No document length consideration
  Example: Long documen

In [35]:
# @title Document Summary Index Retrievers
print ("=" * 60)
print ("🔧 Document Summary Index Retrievers")
print ("=" * 60)

document_summary_retriever_llm = DocumentSummaryIndexLLMRetriever(
    lab.document_summary_index,
    choice_top_k = 3
)

document_summary_retriever_embedding = DocumentSummaryIndexEmbeddingRetriever(
    lab.document_summary_index,
    similarity_top_k = 3
)

query = DEMO_QUERIES["learning_types"]

print(f"Query: {query}")

print("\nA) LLM-based Document Summary Retriever:")
print("Uses LLM to select relevant documents based on summaries")

try:
  nodes_llm = document_summary_retriever_llm.retrieve(query)
  print(f"Retrieved {len(nodes_llm)} nodes")
  for i, node in enumerate(nodes_llm[:2],1):
    print(f"{i}. Score: {node.score:.4f}" if hasattr(node, 'score') and node.score else f"{i}. (Document summary)")
    print(f"   Text: {node.text[:80]}...")
    print()

except Exception as e:
  print(f"LLM-based retrieval demo: {str(e)[:100]}...")

print("B) Embedding-based Document Summary Retriever:")
print("Uses vector similarity between query and document summaries")
try:
    nodes_emb = document_summary_retriever_embedding.retrieve(query)
    print(f"Retrieved {len(nodes_emb)} nodes")
    for i, node in enumerate(nodes_emb[:2], 1):
        print(f"{i}. Score: {node.score:.4f}" if hasattr(node, 'score') and node.score else f"{i}. (Document summary)")
        print(f"   Text: {node.text[:80]}...")
        print()
except Exception as e:
    print(f"Embedding-based retrieval demo: {str(e)[:100]}...")

print("Document Summary Index workflow:")
print("1. Generates summaries for each document using LLM")
print("2. Uses summaries to select relevant documents")
print("3. Returns full content from selected documents")



🔧 Document Summary Index Retrievers
Query: different types of learning

A) LLM-based Document Summary Retriever:
Uses LLM to select relevant documents based on summaries
Retrieved 3 nodes
1. Score: 9.0000
   Text: Machine learning is a subset of artificial intelligence that focuses on algorith...

2. Score: 8.0000
   Text: Deep learning uses neural networks with multiple layers to model and understand ...

B) Embedding-based Document Summary Retriever:
Uses vector similarity between query and document summaries
Retrieved 3 nodes
1. (Document summary)
   Text: Unsupervised learning finds hidden patterns in data without labeled examples....

2. (Document summary)
   Text: Supervised learning uses labeled training data to learn a mapping from inputs to...

Document Summary Index workflow:
1. Generates summaries for each document using LLM
2. Uses summaries to select relevant documents
3. Returns full content from selected documents


In [38]:
# @title Auto Merging Retriever - Hierarchical Context Preservation
print ("=" * 60)
print ("🔧 Auto Merging Retriever")
print ("=" * 60)

node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes = [512, 256 , 128]
)

hier_nodes = node_parser.get_nodes_from_documents(lab.documents)

docstore = SimpleDocumentStore()
docstore.add_documents(hier_nodes)

storage_context = StorageContext.from_defaults(docstore = docstore)

base_index = VectorStoreIndex(hier_nodes, storage_context = storage_context)
base_retriever = base_index.as_retriever(similarity_top_k = 6)

auto_merging_retriever = AutoMergingRetriever(
    base_retriever,
    node_parser,
    verbose = True
)


query = DEMO_QUERIES["advanced"]  # "How do neural networks work in deep learning?"
nodes = auto_merging_retriever.retrieve(query)

print(f"Query: {query}")
print(f"Auto-merged to {len(nodes)} nodes")
for i, node in enumerate(nodes[:3], 1):
    print(f"{i}. Score: {node.score:.4f}" if hasattr(node, 'score') and node.score else f"{i}. (Auto-merged)")
    print(f"   Text: {node.text[:120]}...")
    print()

🔧 Auto Merging Retriever
Query: How do neural networks work in deep learning?
Auto-merged to 2 nodes
1. Score: 0.8570
   Text: Deep learning uses neural networks with multiple layers to model and understand complex patterns in data....

2. Score: 0.6956
   Text: Supervised learning uses labeled training data to learn a mapping from inputs to outputs....



In [40]:
# @title Recursive Retriever
print ("=" * 60)
print ("🔧 Recursive Retriever")
print ("=" * 60)


docs_with_refs = []
for i, doc in enumerate(lab.documents):
    ref_doc = Document(
        text = doc.text,
        metadata = {
            "doc_id" : f"doc_{i}",
            "references" : [f"doc_{j}" for j in range(len(lab.documents)) if j != i][:2]
        }
    )
    docs_with_refs.append(ref_doc)


ref_index = VectorStoreIndex.from_documents(docs_with_refs)

retriever_dict = {
    f"doc_{i}": ref_index.as_retriever(similarity_top_k=1)
    for i in range(len(docs_with_refs))
}

base_retriever = ref_index.as_retriever(similarity_top_k=2)

retriever_dict["vector"] = base_retriever

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict=retriever_dict,
    query_engine_dict={},
    verbose=True
)

query = DEMO_QUERIES["applications"]  # "What are the applications of AI?"


nodes = recursive_retriever.retrieve(query)
print(f"Query: {query}")
print(f"Recursively retrieved {len(nodes)} nodes")

for i, node in enumerate(nodes[:3], 1):
  print(f"{i}. Score: {node.score:.4f}" if hasattr(node, 'score') and node.score else f"{i}. (Recursive)")
  print(f"   Text: {node.text[:100]}...")
  print()


🔧 Recursive Retriever
Retrieving with query id None: What are the applications of AI?
Retrieving text node: Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn from data.
Retrieving text node: Natural language processing enables computers to understand, interpret, and generate human language.
Query: What are the applications of AI?
Recursively retrieved 2 nodes
1. Score: 0.6907
   Text: Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn fr...

2. Score: 0.6500
   Text: Natural language processing enables computers to understand, interpret, and generate human language....



In [42]:
# @title Query Fusion Retriever
print("=" * 60)
print("6. QUERY FUSION RETRIEVER - OVERVIEW")
print("=" * 60)

# Create base retriever
base_retriever = lab.vector_index.as_retriever(similarity_top_k=3)

query = DEMO_QUERIES["comprehensive"]  # "What are the main approaches to machine learning?"
print(f"Query: {query}")
print("QueryFusionRetriever generates multiple query variations and fuses results")
print("using one of three sophisticated fusion modes.")

print("\nOverview of Fusion Modes:")
print("1. RECIPROCAL_RERANK: Uses reciprocal rank fusion (most robust)")
print("2. RELATIVE_SCORE: Preserves score magnitudes (most interpretable)")
print("3. DIST_BASED_SCORE: Statistical normalization (most sophisticated)")

print("\nDemonstration workflow:")
print("Each subsection below explores one fusion mode in detail with:")
print("- Theoretical explanation of the fusion method")
print("- Live demonstration using QueryFusionRetriever")
print("- Manual implementation showing the underlying mathematics")
print("- Use case recommendations and trade-offs")

print(f"\nUsing consistent test query throughout: '{query}'")
print("This allows direct comparison of how each fusion mode handles the same input.")

print("\nProceed to subsections 6.1, 6.2, and 6.3 for detailed demonstrations...")

6. QUERY FUSION RETRIEVER - OVERVIEW
Query: What are the main approaches to machine learning?
QueryFusionRetriever generates multiple query variations and fuses results
using one of three sophisticated fusion modes.

Overview of Fusion Modes:
1. RECIPROCAL_RERANK: Uses reciprocal rank fusion (most robust)
2. RELATIVE_SCORE: Preserves score magnitudes (most interpretable)
3. DIST_BASED_SCORE: Statistical normalization (most sophisticated)

Demonstration workflow:
Each subsection below explores one fusion mode in detail with:
- Theoretical explanation of the fusion method
- Live demonstration using QueryFusionRetriever
- Manual implementation showing the underlying mathematics
- Use case recommendations and trade-offs

Using consistent test query throughout: 'What are the main approaches to machine learning?'
This allows direct comparison of how each fusion mode handles the same input.

Proceed to subsections 6.1, 6.2, and 6.3 for detailed demonstrations...


In [43]:
# @title RECIPROCAL RANK FUSION
print("=" * 60)
print("6.1 RECIPROCAL RANK FUSION MODE DEMONSTRATION")
print("=" * 60)


base_retriever = lab.vector_index.as_retriever(similarity_top_k=3)
print("Testing QueryFusionRetriever with reciprocal_rerank mode:")
print("This demonstrates how RRF works within the query fusion framework")


query = DEMO_QUERIES["comprehensive"]  # "What are the main approaches to machine learning?"

rrf_query_fusion = QueryFusionRetriever(
        [base_retriever],
        similarity_top_k=3,
        num_queries=3,
        mode="reciprocal_rerank",
        use_async=False,
        verbose=True
    )

print(f"\nQuery: {query}")
print("QueryFusionRetriever will:")
print("1. Generate query variations using LLM")
print("2. Retrieve results for each variation")
print("3. Apply Reciprocal Rank Fusion")

nodes = rrf_query_fusion.retrieve(query)

print(f"\nRRF Query Fusion Results:")
for i, node in enumerate(nodes, 1):
    print(f"{i}. Final RRF Score: {node.score:.4f}")
    print(f"   Text: {node.text[:100]}...")
    print()

print("RRF Benefits in Query Fusion Context:")
print("- Automatically handles query variations of different quality")
print("- No bias toward queries that return higher raw scores")
print("- Stable performance across diverse query formulations")

6.1 RECIPROCAL RANK FUSION MODE DEMONSTRATION
Testing QueryFusionRetriever with reciprocal_rerank mode:
This demonstrates how RRF works within the query fusion framework

Query: What are the main approaches to machine learning?
QueryFusionRetriever will:
1. Generate query variations using LLM
2. Retrieve results for each variation
3. Apply Reciprocal Rank Fusion
Generated queries:
1. What are the differences between supervised and unsupervised machine learning?
2. How do reinforcement learning algorithms work in the context of machine learning?

RRF Query Fusion Results:
1. Final RRF Score: 0.0492
   Text: Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn fr...

2. Final RRF Score: 0.0492
   Text: Supervised learning uses labeled training data to learn a mapping from inputs to outputs....

3. Final RRF Score: 0.0167
   Text: Reinforcement learning is a type of machine learning where agents learn to make decisions through re...

RRF Benefi

In [44]:
# @title RELATIVE SCORE FUSION

print("=" * 60)
print("6.2 RELATIVE SCORE FUSION MODE DEMONSTRATION")
print("=" * 60)

base_retriever = lab.vector_index.as_retriever(similarity_top_k=5)

print("Testing QueryFusionRetriever with relative_score mode:")
print("This mode preserves score magnitudes while normalizing across query variations")

# Use the same query for consistency across all fusion modes
query = DEMO_QUERIES["comprehensive"]  # "What are the main approaches to machine learning?"

try:
    # Create query fusion retriever with relative score mode
    rel_score_fusion = QueryFusionRetriever(
        [base_retriever],
        similarity_top_k=3,
        num_queries=3,
        mode="relative_score",
        use_async=False,
        verbose=False
    )

    print(f"\nQuery: {query}")
    print("QueryFusionRetriever with relative_score will:")
    print("1. Generate query variations")
    print("2. Normalize scores within each variation (score/max_score)")
    print("3. Combine normalized scores")

    nodes = rel_score_fusion.retrieve(query)

    print(f"\nRelative Score Fusion Results:")
    for i, node in enumerate(nodes, 1):
        print(f"{i}. Combined Relative Score: {node.score:.4f}")
        print(f"   Text: {node.text[:100]}...")
        print()

    print("Relative Score Benefits in Query Fusion:")
    print("- Preserves confidence information from embedding model")
    print("- Ensures fair contribution from each query variation")
    print("- More interpretable than rank-only methods")

except Exception as e:
    print(f"QueryFusionRetriever error: {e}")
    print("Demonstrating Relative Score concept manually...")

    # Manual demonstration with query variations derived from the main query
    query_variations = [
        DEMO_QUERIES["comprehensive"],  # Original query
        "machine learning approaches and methods",
        "different ML techniques and algorithms"
    ]

    print("Manual Relative Score Fusion with Query Variations:")
    all_results = {}
    query_max_scores = []

    # Step 1: Get results and find max scores for each query
    for i, query_var in enumerate(query_variations):
        print(f"\nQuery variation {i+1}: {query_var}")
        nodes = base_retriever.retrieve(query_var)
        scores = [node.score or 0 for node in nodes]
        max_score = max(scores) if scores else 1.0
        query_max_scores.append(max_score)

        print(f"Max score for this query: {max_score:.4f}")

        # Store results with normalization info
        for node in nodes:
            node_id = node.node.node_id
            original_score = node.score or 0
            normalized_score = original_score / max_score if max_score > 0 else 0

            if node_id not in all_results:
                all_results[node_id] = {
                    'node': node,
                    'combined_score': 0,
                    'contributions': []
                }

            all_results[node_id]['combined_score'] += normalized_score
            all_results[node_id]['contributions'].append({
                'query': i,
                'original': original_score,
                'normalized': normalized_score
            })

    # Step 2: Sort by combined relative score
    sorted_results = sorted(
        all_results.values(),
        key=lambda x: x['combined_score'],
        reverse=True
    )

    print(f"\nCombined Relative Score Results (top 3):")
    for i, result in enumerate(sorted_results[:3], 1):
        print(f"{i}. Combined Score: {result['combined_score']:.4f}")
        print(f"   Score breakdown:")
        for contrib in result['contributions']:
            print(f"     Query {contrib['query']}: {contrib['original']:.3f} → {contrib['normalized']:.3f}")
        print(f"   Text: {result['node'].text[:100]}...")
        print()

    print("Relative Score Normalization Process:")
    print("1. For each query variation, find max_score")
    print("2. Normalize: normalized_score = original_score / max_score")
    print("3. Sum normalized scores across all query variations")
    print("4. Documents with consistently high scores across queries win")

6.2 RELATIVE SCORE FUSION MODE DEMONSTRATION
Testing QueryFusionRetriever with relative_score mode:
This mode preserves score magnitudes while normalizing across query variations

Query: What are the main approaches to machine learning?
QueryFusionRetriever with relative_score will:
1. Generate query variations
2. Normalize scores within each variation (score/max_score)
3. Combine normalized scores

Relative Score Fusion Results:
1. Combined Relative Score: 0.7584
   Text: Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn fr...

2. Combined Relative Score: 0.5261
   Text: Supervised learning uses labeled training data to learn a mapping from inputs to outputs....

3. Combined Relative Score: 0.3650
   Text: Reinforcement learning is a type of machine learning where agents learn to make decisions through re...

Relative Score Benefits in Query Fusion:
- Preserves confidence information from embedding model
- Ensures fair contribution from 

In [46]:
# @title DISTRIBUTION-BASED SCORE FUSION

print("=" * 60)#https://docs.llamaindex.ai/en/stable/examples/retrievers/relative_score_dist_fusion/
print("6.3 DISTRIBUTION-BASED SCORE FUSION MODE DEMONSTRATION")
print("=" * 60)

base_retriever = lab.vector_index.as_retriever(similarity_top_k=8)

print("Testing QueryFusionRetriever with dist_based_score mode:")
print("This mode uses statistical analysis for the most sophisticated score fusion")

# Use the same query for consistency across all fusion modes
query = DEMO_QUERIES["comprehensive"]  # "What are the main approaches to machine learning?"

try:
    # Create query fusion retriever with distribution-based mode
    dist_fusion = QueryFusionRetriever(
        [base_retriever],
        similarity_top_k=3,
        num_queries=3,
        mode="dist_based_score",
        use_async=False,
        verbose=False
    )

    print(f"\nQuery: {query}")
    print("QueryFusionRetriever with dist_based_score will:")
    print("1. Generate query variations")
    print("2. Analyze score distributions for each variation")
    print("3. Apply statistical normalization (z-score, percentiles)")
    print("4. Combine with distribution-aware weighting")

    nodes = dist_fusion.retrieve(query)

    print(f"\nDistribution-Based Fusion Results:")
    for i, node in enumerate(nodes, 1):
        print(f"{i}. Statistically Normalized Score: {node.score:.4f}")
        print(f"   Text: {node.text[:100]}...")
        print()

    print("Distribution-Based Benefits in Query Fusion:")
    print("- Accounts for score distribution differences between query variations")
    print("- Statistically robust against outliers and noise")
    print("- Adapts weighting based on query variation reliability")

except Exception as e:
    print(f"QueryFusionRetriever error: {e}")
    print("Demonstrating Distribution-Based concept manually...")

    if not SCIPY_AVAILABLE:
        print("⚠️ Full statistical analysis requires scipy")

    # Manual demonstration with query variations derived from the main query
    query_variations = [
        DEMO_QUERIES["comprehensive"],  # Original query
        "machine learning approaches and methods",
        "different ML techniques and algorithms"
    ]

    print("Manual Distribution-Based Fusion with Query Variations:")
    all_results = {}
    variation_stats = []

    # Step 1: Collect results and analyze distributions
    for i, query_var in enumerate(query_variations):
        print(f"\nQuery variation {i+1}: {query_var}")
        nodes = base_retriever.retrieve(query_var)
        scores = [node.score or 0 for node in nodes]

        # Calculate distribution statistics
        mean_score = np.mean(scores) if scores else 0
        std_score = np.std(scores) if len(scores) > 1 else 1
        min_score = np.min(scores) if scores else 0
        max_score = np.max(scores) if scores else 1

        stats_info = {
            'mean': mean_score,
            'std': std_score,
            'min': min_score,
            'max': max_score,
            'nodes': nodes,
            'scores': scores
        }
        variation_stats.append(stats_info)

        print(f"Distribution stats: mean={mean_score:.3f}, std={std_score:.3f}")
        print(f"Score range: [{min_score:.3f}, {max_score:.3f}]")

        # Apply z-score normalization
        for node, score in zip(nodes, scores):
            node_id = node.node.node_id

            # Z-score normalization
            if std_score > 0:
                z_score = (score - mean_score) / std_score
            else:
                z_score = 0

            # Convert to [0,1] using sigmoid
            normalized_score = 1 / (1 + np.exp(-z_score))

            if node_id not in all_results:
                all_results[node_id] = {
                    'node': node,
                    'combined_score': 0,
                    'contributions': []
                }

            all_results[node_id]['combined_score'] += normalized_score
            all_results[node_id]['contributions'].append({
                'query': i,
                'original': score,
                'z_score': z_score,
                'normalized': normalized_score
            })

    # Step 2: Sort by combined distribution-based score
    sorted_results = sorted(
        all_results.values(),
        key=lambda x: x['combined_score'],
        reverse=True
    )

    print(f"\nCombined Distribution-Based Results (top 3):")
    for i, result in enumerate(sorted_results[:3], 1):
        print(f"{i}. Combined Score: {result['combined_score']:.4f}")
        print(f"   Statistical breakdown:")
        for contrib in result['contributions']:
            print(f"     Query {contrib['query']}: {contrib['original']:.3f} → "
                  f"z={contrib['z_score']:.2f} → {contrib['normalized']:.3f}")
        print(f"   Text: {result['node'].text[:100]}...")
        print()

    print("Distribution-Based Process:")
    print("1. Calculate mean and std for each query variation")
    print("2. Z-score normalize: z = (score - mean) / std")
    print("3. Sigmoid transform: normalized = 1 / (1 + exp(-z))")
    print("4. Sum normalized scores across variations")
    print("5. Results reflect statistical significance across all query forms")

# Show fusion mode comparison summary
print("\n" + "=" * 60)
print("FUSION MODES COMPARISON SUMMARY")
print("=" * 60)
print("All three modes tested with the same query for direct comparison:")
print(f"Query: {query}")
print()
print("Mode Characteristics:")
print("• RRF (reciprocal_rerank): Most robust, rank-based, scale-invariant")
print("• Relative Score: Preserves confidence, normalizes by max score")
print("• Distribution-Based: Most sophisticated, statistical normalization")
print()
print("Choose based on your use case:")
print("- Production stability → RRF")
print("- Score interpretability → Relative Score")
print("- Statistical robustness → Distribution-Based")

6.3 DISTRIBUTION-BASED SCORE FUSION MODE DEMONSTRATION
Testing QueryFusionRetriever with dist_based_score mode:
This mode uses statistical analysis for the most sophisticated score fusion

Query: What are the main approaches to machine learning?
QueryFusionRetriever with dist_based_score will:
1. Generate query variations
2. Analyze score distributions for each variation
3. Apply statistical normalization (z-score, percentiles)
4. Combine with distribution-aware weighting

Distribution-Based Fusion Results:
1. Statistically Normalized Score: 0.7287
   Text: Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn fr...

2. Statistically Normalized Score: 0.6149
   Text: Supervised learning uses labeled training data to learn a mapping from inputs to outputs....

3. Statistically Normalized Score: 0.5632
   Text: Reinforcement learning is a type of machine learning where agents learn to make decisions through re...

Distribution-Based Benefits in